In [1]:
#Imports necessários
import collections
import numpy as np
import pandas as pds

from imblearn.over_sampling import SMOTE
from pandas import concat
from sklearn import preprocessing

In [2]:
#Análise do dataset
data = pds.read_csv('medical.csv')
print(len(data))
data.head()
#data.describe()
#collections.Counter(data['No-show'])
#data.isnull().any()

110527


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [3]:
#Transformação dos dados
data['ScheduledMonth'] = data.ScheduledDay.str.slice(5,7)
data.ScheduledMonth = preprocessing.scale(data.ScheduledMonth)
data['ScheduledMonthDay'] = data.ScheduledDay.str.slice(8,10)
data.ScheduledMonthDay = preprocessing.scale(data.ScheduledMonthDay)
data.ScheduledDay = pds.to_datetime(data.ScheduledDay.str.slice(0,10))
data.AppointmentDay = pds.to_datetime(data.AppointmentDay.str.slice(0,10))
data.ScheduledDay = abs((data.ScheduledDay - data.AppointmentDay).dt.days)
data.rename(columns={'ScheduledDay':'WaitingDays'}, inplace=True)

data = data.drop('AppointmentDay',1)
data = data.drop('Neighbourhood',1)
data = data.drop('PatientId',1)
data = data.drop('AppointmentID',1)
data.rename(columns={'No-show':'Noshow'}, inplace=True)

data.Age = pds.to_numeric(data.Age)
data.Age = preprocessing.scale(data.Age)
data.Noshow = data.Noshow.map({'Yes': 1, 'No': 0})
data.Gender = data.Gender.map({'F': 1, 'M': 0})
data = data[data.Age > 0]
data.head()

/home/DAITAN/ftsantos/.conda/envs/jupyter/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/DAITAN/ftsantos/.conda/envs/jupyter/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


,Gender,WaitingDays,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Noshow,ScheduledMonth,ScheduledMonthDay
0,1,0,1.077932,0,1,0,0,0,0,0,-1.150983,1.578047
1,0,0,0.818306,0,0,0,0,0,0,0,-1.150983,1.578047
2,1,0,1.077932,0,0,0,0,0,0,0,-1.150983,1.578047
4,1,0,0.818306,0,1,1,0,0,0,0,-1.150983,1.578047
5,1,2,1.683728,0,1,0,0,0,0,0,-1.150983,1.364042


In [4]:
#Verificação de dados do dataset
collections.Counter(data.Noshow)

#Imbalancement treatment com under sampling
#dataNoShow = data.loc[data['Noshow']==1]
#collections.Counter(dataNoShow.Noshow)

#dataShow = data.loc[data['Noshow']==0]
#collections.Counter(dataShow.Noshow)

#data = concat([dataShow.sample(n=25000),dataNoShow])
#collections.Counter(data.Noshow)


Counter({0: 44852, 1: 9558})

In [5]:
#Separação do dataset
from sklearn.model_selection import train_test_split

X = data.drop('Noshow',1)
y = data.Noshow

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [6]:
#Imbalancement treatment com over sampling
balancer = SMOTE()

x_resampled, y_resampled = balancer.fit_sample(X_train, y_train)

print(collections.Counter(y_train))

print(collections.Counter(y_resampled))

Counter({0: 35864, 1: 7664})
Counter({0: 35864, 1: 35864})


In [7]:
#Criação dos modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier  

rf = RandomForestClassifier(n_estimators=100, criterion='entropy', max_features='auto', random_state=0)
lr = LogisticRegression(random_state=0, penalty='l2', C=1, fit_intercept=True, solver='liblinear')
dt = DecisionTreeClassifier(random_state=0, criterion='entropy', splitter='best')  

algs = []
algs.append(rf)
algs.append(lr)
algs.append(dt)

for alg in algs:
    print('Fitting: ', type(alg).__name__)    
    alg.fit(x_resampled, y_resampled)  

/home/DAITAN/ftsantos/.conda/envs/jupyter/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Fitting:  RandomForestClassifier
Fitting:  LogisticRegression
Fitting:  DecisionTreeClassifier


In [8]:
#Acurácia do modelo
from sklearn.metrics import accuracy_score, confusion_matrix

for alg in algs:
    predicted = alg.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    print('Acc: ', type(alg).__name__, f' = {accuracy:.3}')
    print(confusion_matrix(y_test, predicted))
    print()
    print()

Acc:  RandomForestClassifier  = 0.756
[[7738 1250]
 [1401  493]]


Acc:  LogisticRegression  = 0.652
[[6056 2932]
 [ 857 1037]]


Acc:  DecisionTreeClassifier  = 0.755
[[7703 1285]
 [1384  510]]


